# Test de modèles


## <span style="color:lightblue">Installation des dépendances</span>


In [10]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## <span style="color:lightblue">Importation des dépendances</span>


In [11]:
import json
from ultralytics import YOLO
import torch

## <span style="color:lightblue">Environnement</span>


In [12]:
DATASET_PATH = "../data/dataset4"
MODEL_NAME = "yolo11n_trained_openvino_model"
PLATFROM = "i7-11800H"

## <span style="color:lightblue">Code principale</span>


In [13]:
with open(f"{DATASET_PATH}/test/result.json", 'r') as f:
    coco_data = json.load(f)

images = coco_data["images"]

model = YOLO(f"./saved/{MODEL_NAME}")

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

predictions = []

for image in images:
    results = model.predict(f"{DATASET_PATH}/test/{image['file_name']}", device=device)
    
    inference_time = results[0].speed["inference"]
    
    for result in results:
        if result.boxes:
            for box in result.boxes:
                cls = box.cls.item()
                conf = box.conf.item()
                boxes = box.xyxy.tolist()[0]
                predictions.append({
                    "image_id": image["id"],
                    "category_id": int(cls),
                    "bbox": [boxes[0], boxes[1], boxes[2] - boxes[0], boxes[3] - boxes[1]],
                    "score": conf,
                    "inference_time": inference_time
                })

with open(f"./predictions/predictions_{PLATFROM}_{MODEL_NAME}.json", 'w') as f:
    json.dump(predictions, f)

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Device: cpu
Loading saved\yolo11n_trained_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference...

image 1/1 c:\Users\Colin\Documents\M2\Projet_developpement_logiciel_application_IA_embarquee\yolo\..\data\dataset4\test\images\194e378c-608.png: 640x640 1 speed30, 25.0ms
Speed: 32.0ms preprocess, 25.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Colin\Documents\M2\Projet_developpement_logiciel_application_IA_embarquee\yolo\..\data\dataset4\test\images\fcfa48ab-589.png: 640x640 (no detections), 17.0ms
Speed: 3.0ms preprocess, 17.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Colin\Documents\M2\Projet_developpement_logiciel_application_IA_embarquee\yolo\..\data\dataset4\test\images\986425fd-590.png: 

: 